In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg

from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)
sa_dask = dg.from_geopandas(sa, npartitions=4)

del sa

CPU times: user 193 ms, sys: 20.7 ms, total: 214 ms
Wall time: 422 ms


### pga data

In [3]:
%%time
pga = dd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193")
pga_dask = dg.from_geopandas(pgaGeo, npartitions=4)


CPU times: user 651 ms, sys: 308 ms, total: 959 ms
Wall time: 1.71 s


# Merge SA data with PGA events (dask-geodataframe)

In [ ]:
type(pga_dask)

In [5]:
%%time
from spatialpandas import sjoin
pga_with_saData = sjoin(pga_dask, sa_dask)
# pga_with_saData = dg.sjoin(pgaGeo, sa_dask, op="within")

ValueError: `left_df` must be a spatialpandas.GeoDataFrame or spatialpandas.dask.DaskGeoDataFrame
    Received value of type: <class 'dask_geopandas.core.GeoSeries'>

In [ ]:
%%time
PGA = dg.sjoin(pga_dask, sa_dask)#, op="within")